Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50098)
clean_all_dbs(speos.client)

# Create project

In [ ]:
p = script.Project(speos=speos)
print(p)

# Create features
## Source

In [ ]:
source1 = p.create_source(name="Source.1")
source1.set_surface()
source1.commit()

## Sensor

In [ ]:
sensor1 = p.create_sensor(name="Sensor.1")
sensor1.set_irradiance()
sensor1.commit()

## Optical property

In [ ]:
opt_prop1 = p.create_optical_property(name="Material.1")
opt_prop1.commit()

# Read project

In [ ]:
print(p)

# Find a feature
## From feature name  
If no feature is found, an empty list is returned.

In [ ]:
features = p.find(name="UnexistingName")
print(features)

In [ ]:
features = p.find(name="Sensor.1")
print(features[0])

## Filter by feature type 

In [ ]:
# Here wrong type is given : no source is called Sensor.1 in the project
features = p.find(name="Sensor.1", feature_type=script.Source)
print(features)

In [ ]:
features = p.find(name="Sensor.1", feature_type=script.Sensor)
print(features[0])

Possibility to precise the type of sensor, source, simulation

In [ ]:
features = p.find(name="Sensor.1", feature_type=script.Sensor.Irradiance)
print(features[0]._name)

## Using regex

In [ ]:
features = p.find(name="Mat.*", name_regex=True)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

In [ ]:
features = p.find(name=".*", name_regex=True)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

# Delete
This erases the scene content in server database.  
This delete also each feature of the project

In [ ]:
p.delete()
print(p)

In [ ]:
# As the features were deleted just above -> this returns an empty vector
print(p.find(name="Sensor.1"))

# Specificity: Create project from speos file

In [ ]:
p2 = script.Project(speos=speos, path=os.path.join(tests_data_path, "LG_50M_Colorimetric_short.sv5", "LG_50M_Colorimetric_short.sv5"))
print(p2)

Preview

In [ ]:
from ansys.speos.workflow.preview import preview
preview(project=p2)

Possibility to retrieve features as usual

In [ ]:
features = p2.find(name=".*", name_regex=True, feature_type=script.Source)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)
src = features[1]


And modify them

For example here we modify the source spectrum as monochromatic using a wavelength as 550nm (green).

In [ ]:
src.set_surface().set_spectrum().set_monochromatic(wavelength=550)
src.commit()

Retrieve simulation + compute + watch result

In [ ]:
features = p2.find(name=".*", name_regex=True, feature_type=script.Simulation)

sim_feat = features[0]
print(sim_feat)

In [ ]:
sim_feat.compute_CPU()

In [ ]:
from ansys.speos.workflow.open_result import open_result_image

open_result_image(simulation_feature=sim_feat, result_name="ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp")

# Cleanup

In [ ]:
clean_all_dbs(speos.client)